# Metadata Use Cases

The user can add arbitrary metadata fields. Only a handful are required (owner, group, beamline_id). The metadata can be a number, a string, a list, or a dictionary with any amount of nesting. These custom fields are stored in the RunStart Document as siblings of the required fields, and the Data Broker can search on them.

In [0]:
from bluesky import RunEngine, Msg

## The metadata validator

### Let's trip the validator

In [0]:
plan = [Msg('open_run'), Msg('close_run')]  # a pointless plan

In [0]:
RE = RunEngine({})

In [0]:
RE(plan)

### What happens if replace the default validator with one that does nothing?

In [0]:
def permissive_validator(md):
    pass

In [0]:
RE.md_validator = permissive_validator

In [0]:
RE(plan)

We tripped the hard-coded Document validatation. There is no getting around this. (If there were, the errors would just occur all the way down in mongo!)

The moral: Your can use a custom metadata validator to be *more* strict, but there is no point in trying to make it less strict.

In [0]:
RE.md = {'owner': 'demo', 'group': 'demo', 'beamline_id': 'demo'}

### Now use the validator to check for some beamline- or experiment-specific metadata.

In [0]:
def color_validator(md):
    if 'color' not in md:
        raise ValueError("You must specify a color.")
        
RE.md_validator = color_validator

This will trip the validator:

In [0]:
RE(plan)

## Three ways to set metadata

### We can set the metadata when we run a scan.

In [0]:
RE(plan, color='red')

It does not persist between runs.

In [0]:
RE(plan)

### We can add the metadata to the RunEngine's metadata dictionary, which is reused for subsequent runs.

In [0]:
RE.md['color'] = 'red'

In [0]:
RE(scan)  # this picks up color=red from RE.md

Now that `color` is in `RE.md`, it is updated if we specify it.

In [0]:
RE(scan, color='blue')  # this updates RE.md['color'] as a side effect

In [0]:
RE.md['color']

To stop persisting color between runs, delete it.

In [0]:
del RE.md['color']

And now this will error again:

In [0]:
RE(plan)

### It is also possible to associated metadata with the plan itself.

This is rarely necessary.

In [0]:
plan_with_md = [Msg('open_run', color='green'), Msg('close_run')]

In [0]:
RE(plan_with_md)